# 1. Experiment Fragen an azure open ai zu einem CSV File stellen mit einer lokalen Vector Detenbank


## Experiment mit azure open ai embedding model

In [3]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackManager
from langchain.document_loaders import CSVLoader
from langchain import PromptTemplate, LLMChain
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from pathlib import Path

documents = CSVLoader('./Resources/customers-100.csv').load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024,chunk_overlap=64)
texts = text_splitter.split_documents(documents)
embeddings = AzureOpenAIEmbeddings(model="text-embedding-3-large")

faiss_index = FAISS.from_documents(texts, embeddings)
faiss_index.save_local("./Output/customers-100_faiss_az.db")

callbacks = [StreamingStdOutCallbackHandler()]
model = AzureChatOpenAI(
    azure_deployment = "gpt-4-32k",
    streaming=True,
    callback_manager=BaseCallbackManager([
        StreamingStdOutCallbackHandler()]),
)

# Set your query here manually
question = "Wie lautet die Mail Adresse von Greg Mata"
matched_docs = faiss_index.similarity_search(question, 4)
context = ""
for doc in matched_docs:
    context = context + doc.page_content + " \n\n "

template = """
Please use the following context to answer questions.
Context: {context}
 - -
Question: {question}
Answer: Let's think step by step."""

callback_manager = BaseCallbackManager([StreamingStdOutCallbackHandler()])
prompt = PromptTemplate(template=template, input_variables=["context", "question"]).partial(context=context)
llm_chain = LLMChain(prompt=prompt, llm=model)
print(llm_chain.run(question))

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

## Experiment mit hugging face embedding model

In [1]:
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI, OpenAI, AzureChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackManager
from langchain.document_loaders import CSVLoader
from langchain import PromptTemplate, LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from pathlib import Path

documents = CSVLoader('./Resources/customers-100.csv').load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024,chunk_overlap=64)
texts = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

faiss_index = FAISS.from_documents(texts, embeddings)
faiss_index.save_local("./Output/customers-100_faiss.db")

callbacks = [StreamingStdOutCallbackHandler()]
model = AzureChatOpenAI(
    azure_deployment = "gpt-4-32k",
    streaming=True,
    callback_manager=BaseCallbackManager([
        StreamingStdOutCallbackHandler()]),
)

# Set your query here manually
question = "Wie lautet die Mail Adresse von Greg Mata"
matched_docs = faiss_index.similarity_search(question, 4)
context = ""
for doc in matched_docs:
    context = context + doc.page_content + " \n\n "

template = """
Please use the following context to answer questions.
Context: {context}
 - -
Question: {question}
Answer: Let's think step by step."""

callback_manager = BaseCallbackManager([StreamingStdOutCallbackHandler()])
prompt = PromptTemplate(template=template, input_variables=["context", "question"]).partial(context=context)
llm_chain = LLMChain(prompt=prompt, llm=model)
print(llm_chain.run(question))

/home/kosta/miniconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: To find out the number of rows and columns in the dataframe, I can use the `shape` attribute of the dataframe.
Action: python_repl_ast
Action Input: df.shapeThought: To find out the number of rows and columns in the dataframe, I can use the `shape` attribute of the dataframe.
Action: python_repl_ast
Action Input: df.shape(100, 12)The shape attribute of the dataframe returned a tuple with two values. The first value represents the number of rows and the second value represents the number of columns.
Final Answer: The dataframe has 100 rows and 12 columns.The shape attribute of the dataframe returned a tuple with two values. The first value represents the number of rows and the second value represents the number of columns.
Final Answer: The dataframe has 100 rows and 12 columns.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: To find the email address of Yvonne Farmer, I need to filter the dataframe where the First Nam